In [189]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os 

import sys; sys.path.append('/home/mitch/school/mfl/')
import src.json_utils

In [190]:
raw = '/home/mitch/Dropbox/data/mexico_fls/raw/2005/'
interim = '/home/mitch/Dropbox/data/mexico_fls/interim2/2005/'

figs = '/home/mitch/github/mfl/fig/'
dicts= '/home/mitch/github/mfl/dicts/'

In [191]:
education = src.json_utils.load_json(dicts + 'education.json')
rename = src.json_utils.load_json(dicts + 'rename_ind_2005.json')

In [192]:
ids = ['folio', 'ls', 'pid_link']
keep = ['folio', 'ls', 'pid_link']
os.chdir(raw + 'book IIIB/data/')
portad = pd.read_stata('iiib_portad.dta')
portad = portad.sort_values(by=['folio', 'ls'])

In [193]:
portad.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20607 entries, 0 to 20606
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   catorcena  20607 non-null  float32
 1   edad       20606 non-null  float32
 2   edo_civil  20607 non-null  int8   
 3   ent        20607 non-null  float32
 4   folio      20607 non-null  object 
 5   ls         20607 non-null  object 
 6   pid_link   20607 non-null  object 
 7   rel        20607 non-null  float32
dtypes: float32(4), int8(1), object(3)
memory usage: 986.1+ KB


In [194]:
os.chdir(raw + 'book IIIA/data/')
weights_3a = pd.read_stata('weights.dta')
weights_3a = (weights_3a.query('fac_3a>0')
              .drop_duplicates()
              .sort_values(by=['folio', 'ls']))
weights_3a.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19544 entries, 0 to 35087
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   folio   19544 non-null  object
 1   ls      19544 non-null  object
 2   fac_3a  19544 non-null  int32 
dtypes: int32(1), object(2)
memory usage: 534.4+ KB


In [195]:
# ids
os.chdir(raw + 'book C/data/')
df_ls = pd.read_stata('c_ls.dta')
df_ls = df_ls.drop(columns='pid_link')

os.chdir(raw + 'book IIIA/data/')
# employment
df_tb = pd.read_stata('iiia_tb.dta')
df_tb = df_tb.drop(columns='pid_link')

# education
df_ed = pd.read_stata('iiia_ed.dta')
df_ed = df_ed.drop(columns='pid_link')

# non labor income, is ANNUAL
df_iin = pd.read_stata('iiia_iin.dta')
df_iin = df_iin.drop(columns='pid_link')

os.chdir(raw + 'book IIIB/data/')
# credit
df_cr = pd.read_stata('iiib_cr.dta')
df_cr = df_cr.drop(columns='pid_link')

# credit 1
df_cr1 = pd.read_stata('iiib_cr1.dta')
df_cr1 = df_cr1.drop(columns='pid_link')

# family structure (gender)
df_thi = pd.read_stata('iiib_thi.dta')
df_thi = df_thi.drop(columns='pid_link')

In [196]:
df = (weights_3a.merge(portad, on=['folio', 'ls'], how='left')
        .merge(df_ls, on=['folio', 'ls'], how='left')
        .merge(df_tb, on=['folio', 'ls'], how='left')
        .merge(df_ed, on=['folio', 'ls'], how='left')
        .merge(df_cr, on=['folio', 'ls'], how='left')
        .merge(df_cr1, on=['folio', 'ls'], how='left')
        .merge(df_thi, on=['folio', 'ls'], how='left')
        )

In [197]:
df = df.rename(columns=rename)
df = df[rename.values()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19930 entries, 0 to 19929
Data columns (total 51 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   hhid                        19930 non-null  object 
 1   hhmember                    19930 non-null  object 
 2   weight_ind                  19930 non-null  int32  
 3   is_hhm                      19873 non-null  float32
 4   age                         19463 non-null  float32
 5   state                       19464 non-null  float32
 6   attended_school             19589 non-null  float32
 7   education                   17500 non-null  float32
 8   gender                      19452 non-null  float32
 9   income_type                 6365 non-null   float64
 10  income_undetailed           5048 non-null   float32
 11  has_income_second_job       112 non-null    float32
 12  income_second_job           97 non-null     float32
 13  has_wage                    195

# Education and Gender

In [198]:
# define household size
df['hh_size'] = df.groupby('hhid')['hhmember'].transform('size')

# Education

In [199]:
df['attended_school'] = df['attended_school'].replace({1.0:1.0, 3.0:0.0})
df.attended_school.value_counts()

1.0    17500
0.0     2089
Name: attended_school, dtype: int64

In [200]:
education

{'1.0': 'low',
 '2.0': 'low',
 '3.0': 'low',
 '4.0': 'medium',
 '5.0': 'medium',
 '6.0': 'medium',
 '7.0': 'medium',
 '8.0': 'medium',
 '9.0': 'high',
 '10.0': 'high',
 '98.0': 'low'}

In [201]:
df.education = df.education.astype('str').replace(education)
df.loc[df.attended_school == 0.0, 'education'] = 'low'
df.education.value_counts()

low       9514
medium    8266
high      1809
nan        341
Name: education, dtype: int64

# Gender

In [202]:
df.gender.value_counts()

1.0    10719
3.0     5188
2.0     3545
Name: gender, dtype: int64

In [203]:
df['gender'] = df['gender'].replace(
    {1.0:'Female',
     2.0:'Male',
     3.0:'Male'}
    )
df.gender.value_counts()

Female    10719
Male       8733
Name: gender, dtype: int64

# Income

In [204]:
has_values = {
    'wage':'A',
    'piecework':'B',
    'tips':'C',
    'extrahours':'D',
    'christmasbonus':'E',
    'bonus':'F',
    'profitdist':'H',
    'other':'M'
}

In [205]:
df['income_main_job'] = 0.00

undetailed       = df['income_type'] == 3.0
detailed    = df['income_type'] == 1.0
dk          = df['income_type'] == 8.0

df.loc[undetailed, 'income_main_job'] = df.loc[undetailed, 'income_undetailed']

In [206]:
detailed_income_sources = ['wage', 'piecework', 'tips', 'extrahours', 'christmasbonus', 'bonus', 'profitdist', 'other']
for detailed_income_source in detailed_income_sources:
    knows = df['has_' + detailed_income_source] == has_values[detailed_income_source]
    dk = df['has_' + detailed_income_source] == 8

    df.loc[dk, detailed_income_source] = 0.00
    df[detailed_income_source] = df[detailed_income_source].fillna(0.00)
df.loc[detailed, 'income_main_job'] = df.loc[detailed, detailed_income_sources].sum(axis=1)

In [207]:
income_sources = [
    'income_main_job',
    'income_second_job',
    'income_main_business',
    'income_second_business'
]
df['income'] = df[income_sources].sum(axis=1)

# Debts

In [208]:
df['has_debts'] = df['has_debts'].replace(
    {1.0:1, 8.0:0}
    ).fillna(0)

no_debts = df['has_debts'] == 0
df.loc[no_debts, 'debts'] = 0.0

df['has_debts'] = (df['debts'] > 0) + 0.0
df.has_debts.value_counts()

0.0    14874
1.0     5056
Name: has_debts, dtype: int64

# Loans

In [209]:
df.asked_for_loan_12mth.value_counts()

3.0    5363
1.0    1625
Name: asked_for_loan_12mth, dtype: int64

In [210]:
# more formal measure of debts
df['asked_for_loan_12mth'] = df['asked_for_loan_12mth'].replace({1.0:1, 3.0:0})

loan_locations = {
    1.0:'bank',
    2.0:'savings_fund',
    3.0:'moneylender',
    4.0:'relative',
    5.0:'friends',
    6.0:'work',
    7.0:'pawnshop',
    8.0:'verbal_agreement_credit_program',
    9.0:'other_govt',
    10.0:'other',
    -1.0: np.nan
}
formal = ['bank', 'savings_fund', 'moneylender', 'pawnshop']
informal = ['relative', 'friends', 'work', 'verbal_agreement_credit_program', 'other_govt', 'other']
friends_relatives = ['relative', 'friends']

In [211]:
df['loan_location'] = df['loan_location'].replace(loan_locations)

df['has_loan_formal']    = df['loan_location'].isin(formal)
df['has_loan_informal']  = df['loan_location'].isin(informal)
df['has_loan_friends_relatives']  = df['loan_location'].isin(friends_relatives)

df['has_loan'] = df['has_loan_formal'] | df['has_loan_informal'] | df['has_loan_friends_relatives']

got_loan = df['received_loan'] == 1.0
df.loc[~got_loan, 'loan'] = 0.0

df['loan_formal']   = df['loan'] * (df['has_loan_formal'] + 0.0)
df['loan_informal'] = df['loan'] * (df['has_loan_informal'] + 0.0)
df['loan_friends_relatives']   = df['loan'] * (df['has_loan_friends_relatives'] + 0.0)

# Savings

In [212]:
df.has_savings.value_counts()

3.0    17042
1.0     2156
7.0      127
Name: has_savings, dtype: int64

In [213]:
has_savings = df['has_savings'] == 1.0
no_savings = df['has_savings'] == 3.0
dk_savings = df['has_savings'] == 7.0

#df.loc[has_savings, 'savings'] = df.loc[df.has_savings, 'savings']
df.loc[no_savings, 'savings'] = 0.0
df.loc[dk_savings, 'savings'] = 0.0

In [214]:
df['has_savings_none']              = df['has_savings_none'] == 1.0
df['has_savings_bank']              = df['has_savings_bank'] == 2.0
df['has_savings_coop']              = df['has_savings_coop'] == 3.0
df['has_savings_savingsbank']       = df['has_savings_savingsbank'] == 4.0
df['has_savings_friend_not_hhm']    = df['has_savings_friend_not_hhm'] == 5.0
df['has_savings_afores']            = df['has_savings_afores'] == 6.0
df['has_savings_caja_solidaria']    = df['has_savings_caja_solidaria'] == 7.0
df['has_savings_house']             = df['has_savings_house'] == 8.0
df['has_savings_work']              = df['has_savings_work'] == 9.0
df['has_savings_other']             = df['has_savings_other'] == 10.0

In [215]:
# restrict to people with a well defined age
df = (df.query('age >= 20')
      .query('age <= 65')
)

In [216]:
get_sum = [
    'income',
    'savings',
    'debts'
]

get_any = [
    'has_savings',
    'has_debts', 
    'has_loan',
    'has_loan_formal',
    'has_loan_informal',
    'has_loan_friends_relatives',
    'has_savings_none',
    'has_savings_bank',
    'has_savings_coop',
    'has_savings_savingsbank',
    'has_savings_friend_not_hhm',
    'has_savings_afores',
    'has_savings_caja_solidaria',
    'has_savings_house',
    'has_savings_work',
    'has_savings_other'
]

for var in get_sum:
    df['ind_' + var] = df.groupby('hhid')[var].transform('sum')

for var in get_any:
    df['ind_' + var] = df.groupby('hhid')[var].transform(lambda x : x.sum() > 0)

In [217]:
df = (df.drop(columns = get_sum)
      .drop(columns = get_any))

In [218]:
# create single observation for each household by restricting to household head
df = df[df['is_hhm'] == 1.0].reset_index(drop=True)

# Income Outliers

In [219]:
from statsmodels.stats.weightstats import DescrStatsW
q_995 = DescrStatsW(df['ind_income'], df['weight_ind']).quantile(0.995).iloc[0]
df = df[df['ind_income'] < q_995]

In [220]:
os.chdir(interim)
df.to_csv('ind_2005.csv', index=False)

In [221]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5089 entries, 0 to 5111
Data columns (total 62 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   hhid                            5089 non-null   object 
 1   hhmember                        5089 non-null   object 
 2   weight_ind                      5089 non-null   int32  
 3   is_hhm                          5089 non-null   float32
 4   age                             5089 non-null   float32
 5   state                           5089 non-null   float32
 6   attended_school                 5087 non-null   float32
 7   education                       5089 non-null   object 
 8   gender                          5089 non-null   object 
 9   income_type                     2592 non-null   float64
 10  income_undetailed               2054 non-null   float32
 11  has_income_second_job           54 non-null     float32
 12  income_second_job               45